In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
query="what is a cat ?"
input =  tokenizer(query , return_tensors="pt")


In [12]:
print(input)

{'input_ids': tensor([[ 125,   19,    3,    9, 1712,    3,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [13]:
result = model.generate(**input)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [14]:
answer = tokenizer.decode(result[0], skip_special_tokens=True)
print(answer)

Was ist ein Katz?


In [15]:
# Load model directly from Hugging Face repository
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")



Question: who is the main character of the story ?
Answer: Lily


In [29]:
# List of question-context pairs
question_context_pairs = [
    ("summarise the story ", "Once upon a time, in a quaint village nestled between rolling hills and lush forests, there lived a young girl named Lily. Lily was known throughout the village for her insatiable curiosity and her love for exploring the world around her.One sunny morning, as she was wandering near the edge of the forest, Lily stumbled upon a mysterious old book. Its pages were filled with strange symbols and diagrams.Intrigued, Lily decided to uncover the secrets of the book, setting off on an adventure that would change her life forever.")
    # Add more question-context pairs as needed
]

# Generate answers for each pair
for question, context in question_context_pairs:
    input_text = f"question: {question} context: {context} </s>"
    input_ids = tokenizer.encode(input_text, return_tensors='pt', max_length=300,  truncation=True)
    output = model.generate(input_ids,  max_length=150, min_length=5)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    print("Question:", question)
    print("Answer:", answer)


Question: summarise the story 
Answer: Lily decided to uncover the secrets of the book, setting off on an adventure that would change her life forever


In [31]:
question = "translated English to German : I am good how are you ?"
input_ids = tokenizer.encode(question, return_tensors='pt', max_length=300,  truncation=True)
output = model.generate(input_ids,  max_length=150, min_length=5)
answer = tokenizer.decode(output[0], skip_special_tokens=True)
print("Question:", question)
print("Answer:", answer)

Question: translated English to German : I am good how are you ?
Answer: : Ich bin gut wie sind Sie?


In [35]:
def print_formatted_text(text, max_line_length):
    words = text.split()  # Split the text into words
    current_line = ""

    for word in words:
        if len(current_line) + len(word) + 1 <= max_line_length:
            current_line += " " + word if current_line else word
        else:
            print(current_line)
            current_line = word

    if current_line:
        print(current_line)

answer = "This is a long answer that needs to be split into multiple lines so that each line contains at most 30 characters."
max_line_length = 30
print_formatted_text(answer, max_line_length)

This is a long answer that
needs to be split into
multiple lines so that each
line contains at most 30
characters.


In [40]:
question = """ Summarize : Fine-Tuning the Text-To-Text Transfer Transformer (T5) for Closed-Book Question Answering
Or: What does T5 know?
The following tutorial guides you through the process of fine-tuning a pre-trained T5 model, evaluating its accuracy, and using it for prediction, all on a free Google Cloud TPU Open In Colab.

Background
T5 was introduced in the paper Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer. In that paper, we provided a comprehensive picture of how we pre-trained a standard text-to-text Transformer model on a large text corpus, achieving state-of-the-art results on many NLP tasks after fine-tuning.

We pre-trained T5 on a mixture of supervised and unsupervised tasks with the majoriy of data coming from an unlabeled dataset we developed called C4. C4 is based on a massive scrape of the web produced by Common Crawl. Loosely speaking, pre-training on C4 ideally gives T5 an understanding of natural language in addition to general world knowledge.

How can we assess what T5 knows?
As the name implies, T5 is a text-to-text model, which enables us to train it on arbitrary tasks involving a textual input and output. As we showed in our paper, a huge variety of NLP tasks can be cast in this format, including translation, summarization, and even classification and regression tasks.

One way to use this text-to-text framework is on reading comprehension problems, where the model is fed some context along with a question and is trained to predict the question's answer. For example, we might feed the model the text from the Wikipedia article about Hurrican Connie along with the question "On what date did Hurricane Connie occur?" and train the model to predict the answer "August 3rd, 1955". A related task is open-domain question answering (QA) where the model is not provided with this oracle context. Typically, open-domain QA systems include a mechanism to look up information in an external knowledge source. This setting is similar to an "open-book" exam.

In this notebook, we'll be training T5 on a variant of this task which we call closed-book question answering. In closed-book QA, we feed the model a question without any context or access to external knowledge and train it to predict the answer. Since the model doesn't receive any context, the primary way it can learn to answer these questions is based on the "knowledge" it obtained during pre-training. We don't expect T5 to contain super specific information, so we will be focusing on two question-answering datasets which largely include trivia questions (i.e. facts about well-known subjects). Similar investigations have recently been done to test the knowledge stored by BERT and GPT-2.

T5 was not pre-trained on closed-book QA, so in this notebook we'll first create two new tasks and then use the t5 library to fine-tune, evaluate, and obtain predictions from T5. In the end, T5's performance on closed-book QA can give us a sense of what kind (and how much) information T5 managed to learn during pre-training.

State-of-the-art Results
We published a more in-depth investigation of closed-book QA with T5 where we achieved SOTA on open-domain variants of WebQuestions and TriviaQA in addition to surpisingly strong results on Natural Questions. The code in this notebook is a simplified version of those experiments but still produces good results.

For code to reproduce our best results, please see the t5_closed_book_qa repo.

Caveats
While we provide instructions for running on a Cloud TPU via Colab for free, a Google Cloud Storage (GCS) bucket is required for storing model parameters and data. The GCS free tier provides 5 GB of storage, which should be enough to train the large model and smaller but not the 3B or 11B parameter models. You can use part of your initial $300 credit to get more space.
The Cloud TPU provided by Colab (a v2-8) does not have enough memory to fine-tune the 11B parameter model. For this model, you will need to fine-tune inside of a GCP instance (see README). """
input_ids = tokenizer.encode(question, return_tensors='pt', max_length=300,  truncation=True)
output = model.generate(input_ids,  max_length=550, min_length=5)
answer = tokenizer.decode(output[0], skip_special_tokens=True)
print("Question:", question)
print("***"* 4)
print("\n\nAnswer to question is : \n \n")
print_formatted_text(answer, 100)

Question:  Summarize : Fine-Tuning the Text-To-Text Transfer Transformer (T5) for Closed-Book Question Answering
Or: What does T5 know?
The following tutorial guides you through the process of fine-tuning a pre-trained T5 model, evaluating its accuracy, and using it for prediction, all on a free Google Cloud TPU Open In Colab.

Background
T5 was introduced in the paper Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer. In that paper, we provided a comprehensive picture of how we pre-trained a standard text-to-text Transformer model on a large text corpus, achieving state-of-the-art results on many NLP tasks after fine-tuning.

We pre-trained T5 on a mixture of supervised and unsupervised tasks with the majoriy of data coming from an unlabeled dataset we developed called C4. C4 is based on a massive scrape of the web produced by Common Crawl. Loosely speaking, pre-training on C4 ideally gives T5 an understanding of natural language in addition to general 